## Build Recommend System

### step1：分词

In [2]:
from lib.utils import init_client
import os
import jieba

In [3]:
client = init_client()
db = client.novelRS
coll = db.novels
coll.find({'is_saved': True}).count()

/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


235

In [4]:
coll.find_one({'is_saved': True})['_id']

ObjectId('5e39386540413c24a5becad8')

In [5]:
def get_words(nid):
    filename = os.path.join('..', 'crawler', 'corpus', str(nid) + ".txt")
    with open(filename, "rb") as f:
        text = f.read()
    return text

In [6]:
def save_seg_novel(nid, text):
    text = text.encode("utf-8")
    filename = os.path.join('seg_corpus', str(nid) + ".txt")
    with open(filename, "wb") as f:
        f.write(text)
    coll.update({'_id': nid}, {
        '$set': {
            'is_segment': True
        }
    })

In [7]:
novels = []
for novel in coll.find({'is_saved': True, 'is_segment': False}): # 读入内存，防止cursor找不到
    novels.append(novel)

In [8]:
count = 1
for novel in novels:
    print(count, novel["name"])
    text = get_words(novel["_id"])
    words = []
    i = 0
    for word in list(jieba.cut(text)):
        if len(word) > 1 and word != "\r\n":
            words.append(word)
            i += 1
        if i > 100000: # 每部小说最多10万词
            break
    save_seg_novel(novel["_id"], " ".join(words))
    count += 1

Building prefix dict from the default dictionary ...


1 太古神狱


Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.587 seconds.
Prefix dict has been built successfully.
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


2 魔能星海
3 我铸造了仙界
4 当斗罗大陆回到八千年前
5 镇鼎
6 最强真言道统
7 全能召唤师系统
8 万古帝尊
9 从今天开始捡属性
10 美男天师联盟
11 不可名状的邪神
12 天苍神帝
13 武破九荒
14 混沌天帝诀
15 异世之水浒群星降临
16 异域之系统无敌
17 我爆了亿万BOSS
18 焚天吞星
19 我能复制万族天赋
20 随身携带上古天庭
21 幻兽帝国
22 逍遥醉世录
23 反派至尊
24 太荒吞天诀
25 万千之心
26 洪荒世界之九龙金牌
27 侠女来袭：本王妃你不可
28 踏破太古
29 捕鱼狂帝系统
30 星空蕴道
31 斗罗大陆4终极斗罗
32 儒雅随和的我不是魔头
33 终极斗罗
34 四宇沉浮
35 我是真不想穿越
36 辽辽天地间
37 我的宠物居然变异了
38 超武末世
39 天降鬼才
40 山海临城
41 学神传革命
42 勇者至尊
43 万古最强宗
44 齐天册
45 道祖，我来自地球
46 日常系大侠
47 太古剑尊
48 至尊重生
49 我气哭了百万修炼者
50 最强升级系统
51 垂钓之神
52 香火炼神道
53 我是半妖
54 逆鳞决
55 我在西北开加油站
56 元始魔域
57 天武剑界
58 万古犹存
59 尘梦问逍遥
60 斗武乾坤
61 天陆传说
62 她有一间时空小屋
63 剑主八荒
64 逆流之他们的世界
65 无敌系统之请你砍死我
66 玩家请自重
67 王道寒
68 花都绝品医神
69 第一序列
70 三国之帝霸万界系统
71 无敌懒人系统
72 厉害了我的原始人
73 宁院长的诸天之旅
74 凌霄大圣
75 情不知所深
76 长生榜之凡人纪
77 我的万界刺客系统
78 至尊纹章
79 回到原始社会做酋长
80 成为祖师爷从死后开始
81 末世最强回收系统
82 武魂谭
83 魔法科技大洪流
84 灵元灭世
85 谛魔之渊
86 地球穿越时代
87 浩瀚仙秦
88 冰与火之魔山
89 独步剑武
90 封灵星神
91 灭世剑尊
92 无敌霸帝
93 古神的自我修养
94 神兽召唤系统
95 打造荒古世界
96 异界无双之吕布
97 叩王庭
98 异事笔录
99 伏天道纪
100 他身上有怨气
101 逆武云尊
102 平凡不平凡的世界
103 钥之旅
104 剑行大道
105 梦道破天

### step2：词表分析

In [9]:
# 读取所有小说
novels = []
for novel in coll.find({'is_saved': True}): # 读入内存，防止cursor找不到
    novels.append(novel)

In [10]:
def get_words_from_disk(nid):
    filename = os.path.join('seg_corpus', str(nid) + ".txt")
    with open(filename, "rb") as f:
        text = f.read()
    return text.decode("utf-8").split(' ')

In [11]:
from collections import Counter

counter = Counter()

for i, novel in enumerate(novels):
    print(i+1, novel["name"])
    words = get_words_from_disk(novel["_id"])
    counter.update(words)

1 太古神狱
2 魔能星海
3 我铸造了仙界
4 当斗罗大陆回到八千年前
5 镇鼎
6 最强真言道统
7 全能召唤师系统
8 万古帝尊
9 从今天开始捡属性
10 美男天师联盟
11 不可名状的邪神
12 天苍神帝
13 武破九荒
14 混沌天帝诀
15 异世之水浒群星降临
16 异域之系统无敌
17 我爆了亿万BOSS
18 焚天吞星
19 我能复制万族天赋
20 随身携带上古天庭
21 幻兽帝国
22 逍遥醉世录
23 反派至尊
24 太荒吞天诀
25 万千之心
26 洪荒世界之九龙金牌
27 侠女来袭：本王妃你不可
28 踏破太古
29 捕鱼狂帝系统
30 星空蕴道
31 斗罗大陆4终极斗罗
32 儒雅随和的我不是魔头
33 终极斗罗
34 四宇沉浮
35 我是真不想穿越
36 辽辽天地间
37 我的宠物居然变异了
38 超武末世
39 天降鬼才
40 山海临城
41 学神传革命
42 勇者至尊
43 万古最强宗
44 齐天册
45 道祖，我来自地球
46 日常系大侠
47 太古剑尊
48 至尊重生
49 我气哭了百万修炼者
50 最强升级系统
51 垂钓之神
52 香火炼神道
53 我是半妖
54 逆鳞决
55 我在西北开加油站
56 元始魔域
57 天武剑界
58 万古犹存
59 尘梦问逍遥
60 斗武乾坤
61 天陆传说
62 她有一间时空小屋
63 剑主八荒
64 逆流之他们的世界
65 无敌系统之请你砍死我
66 玩家请自重
67 王道寒
68 花都绝品医神
69 第一序列
70 三国之帝霸万界系统
71 无敌懒人系统
72 厉害了我的原始人
73 宁院长的诸天之旅
74 凌霄大圣
75 情不知所深
76 长生榜之凡人纪
77 我的万界刺客系统
78 至尊纹章
79 回到原始社会做酋长
80 成为祖师爷从死后开始
81 末世最强回收系统
82 武魂谭
83 魔法科技大洪流
84 灵元灭世
85 谛魔之渊
86 地球穿越时代
87 浩瀚仙秦
88 冰与火之魔山
89 独步剑武
90 封灵星神
91 灭世剑尊
92 无敌霸帝
93 古神的自我修养
94 神兽召唤系统
95 打造荒古世界
96 异界无双之吕布
97 叩王庭
98 异事笔录
99 伏天道纪
100 他身上有怨气
101 逆武云尊
102 平凡不平凡的世界
103 钥之旅
104 剑行大道
1

In [12]:
len(counter)

405714

In [13]:
counter.most_common(100)

[('自己', 146384),
 ('没有', 145870),
 ('一个', 135504),
 ('什么', 110345),
 ('他们', 101873),
 ('知道', 85319),
 ('已经', 83837),
 ('说道', 77891),
 ('就是', 76878),
 ('这个', 76771),
 ('我们', 72877),
 ('不是', 65567),
 ('现在', 64080),
 ('可以', 61279),
 ('看着', 59452),
 ('还是', 57523),
 ('不过', 53901),
 ('你们', 52240),
 ('怎么', 51245),
 ('有些', 49764),
 ('起来', 48297),
 ('这么', 48015),
 ('时候', 45847),
 ('只是', 44637),
 ('出来', 43056),
 ('这样', 42440),
 ('但是', 41473),
 ('看到', 41004),
 ('这里', 40511),
 ('可是', 38618),
 ('直接', 38606),
 ('虽然', 38064),
 ('还有', 37582),
 ('不会', 36604),
 ('这些', 36377),
 ('之后', 35647),
 ('因为', 34593),
 ('一声', 33348),
 ('感觉', 33246),
 ('一下', 33090),
 ('可能', 33077),
 ('所以', 32104),
 ('------------', 30295),
 ('那个', 30243),
 ('开始', 30113),
 ('如果', 30105),
 ('的话', 29856),
 ('突然', 29806),
 ('事情', 29289),
 ('出现', 28888),
 ('不能', 28651),
 ('修炼', 28526),
 ('身上', 28075),
 ('实力', 27687),
 ('然后', 27397),
 ('竟然', 27395),
 ('东西', 27253),
 ('时间', 27124),
 ('一些', 26973),
 ('声音', 26947),
 ('之中', 26935),
 ('真的', 2

### step3：TF-IDF向量化

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
contents = [open(os.path.join('seg_corpus', str(novel["_id"]) + ".txt"), "rb") for novel in novels]

In [16]:
vectorizer = TfidfVectorizer(input="file", min_df=0.01)

In [17]:
X = vectorizer.fit_transform(contents)

In [18]:
len(vectorizer.vocabulary_)

113466

In [19]:
vectorizer.vocabulary_

{'作品': 15164,
 '正文': 67153,
 '秦烈': 83236,
 '遇袭': 103432,
 '天龙': 37574,
 '帝国': 45318,
 '位于': 14700,
 '东荒': 6823,
 '大陆': 36947,
 '边陲': 100115,
 '一隅': 3155,
 '东面': 6849,
 '有着': 63641,
 '广袤': 46176,
 '无垠': 60871,
 '海洋': 71434,
 '西北面': 94714,
 '群山环绕': 87879,
 '参天': 26760,
 '大岳': 36355,
 '巍峨': 44444,
 '各种各样': 29195,
 '生灵': 77511,
 '种族': 83102,
 '这片': 101535,
 '名叫': 29594,
 '路上': 98755,
 '无处不在': 60876,
 '灵气': 73674,
 '不断': 5875,
 '淬炼': 71822,
 '自身': 90980,
 '永恒': 69280,
 '追寻着': 102644,
 '天地间': 37165,
 '终极': 86709,
 '玄妙': 76361,
 '清晨': 72137,
 '时分': 61574,
 '云岚': 10695,
 '城北': 34484,
 '一支': 2117,
 '十三': 24700,
 '队伍': 107397,
 '正在': 67080,
 '极寒': 65318,
 '山脉': 44235,
 '古树': 27961,
 '林海': 65458,
 '匆匆': 24396,
 '穿行': 83771,
 '深处': 71877,
 '生长': 77583,
 '不少': 5646,
 '灵草': 73741,
 '活动': 70894,
 '只是': 28186,
 '极少': 65321,
 '有人': 63201,
 '独自': 75963,
 '前往': 23233,
 '这些': 101024,
 '王府': 76663,
 '义子': 8831,
 '炼体': 74110,
 '境界': 34852,
 '年纪': 45900,
 '超过': 98313,
 '十六岁': 24899,
 '三天': 4110,
 '举行': 8760,

In [20]:
import pickle

with open("vectorizer.pickle", "wb") as f:
    pickle.dump(vectorizer, f)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(nid1, nid2):
    """ 计算相似度 """
    f1 = open(os.path.join('seg_corpus', str(nid1) + ".txt"), "rb")
    f2 = open(os.path.join('seg_corpus', str(nid2) + ".txt"), "rb")
    a = vectorizer.transform([f1, f2])
    return cosine_similarity(a)[0,1]

In [22]:
compute_similarity(novels[1]["_id"], novels[2]["_id"])

0.08939962935103628

In [23]:
from bson.objectid import ObjectId

def get_most_similar(nid):
    """ 获取最相似的 """
    sim_map = {}
    for n in novels:
        if n["_id"] != nid:
            sim_map[str(n["_id"])] = compute_similarity(nid, n["_id"])
#             print(n["_id"], sim_map[str(n["_id"])])
            if len(sim_map) > 20: # 多余20个删除最小的
                lowest_id = 0
                lowest = 1
                for k in sim_map.keys():
                    if sim_map[k] < lowest:
                        lowest_id = k
                        lowest = sim_map[k]
                del sim_map[lowest_id]
    res = {}
    for k in sim_map.keys():
        novel = coll.find_one({"_id": ObjectId(k)})
        res[k] = {
            "name": novel["name"],
            "author": novel["author"],
            "sim": sim_map[k],
        }
    return res

In [26]:
coll.find_one({"name": "开局就是神兽"})

{'_id': ObjectId('5e393a2c40413c24a5becbda'),
 'name': '开局就是神兽',
 'url': 'http://www.xqishuta.com/Shtml67940.html',
 'author': '我家蓝胖好凶',
 'category': '玄幻奇幻',
 'abstract': '\n    【【2019    “玄幻王者”征文】参赛作品】开局就是神兽，神兽血脉配合系统，苏源无敌了。\n声明：本站提供《开局就是神兽》TXT全集下载收集整理自网络，仅作为阅读交流之用，并无任何商业目的。版权归作者或出版社所有。如作者、出版社认为本站行为侵权，请联系本站，本站会立即删除您认为侵权的作品。\n',
 'download_url': 'http://dzs.xqishuta.com/67/67940/开局就是神兽.txt',
 'is_downloaded': True,
 'success': True,
 'is_segment': True,
 'is_saved': True}

In [27]:
get_most_similar(ObjectId('5e393a2c40413c24a5becbda')) #时间要很久

{'5e39388f40413c24a5becafd': {'name': '我是真不想穿越',
  'author': '碧蓝的世界',
  'sim': 0.4279563904366569},
 '5e39388f40413c24a5becafe': {'name': '辽辽天地间',
  'author': '人生大乱',
  'sim': 0.4214504454817345},
 '5e3938c940413c24a5becb16': {'name': '天武剑界',
  'author': '武之凉夏',
  'sim': 0.43885092090712063},
 '5e3938c940413c24a5becb18': {'name': '尘梦问逍遥',
  'author': '游梦凡人',
  'sim': 0.42911142208495795},
 '5e3938c940413c24a5becb1b': {'name': '天陆传说',
  'author': '每文哥哥',
  'sim': 0.3889317032117},
 '5e3938e640413c24a5becb26': {'name': '无敌懒人系统',
  'author': '钻石熊',
  'sim': 0.3894758021129443},
 '5e39390d40413c24a5becb2a': {'name': '情不知所深',
  'author': '文小文呀',
  'sim': 0.3832767510001776},
 '5e39393540413c24a5becb4b': {'name': '他身上有怨气',
  'author': '莫忧人',
  'sim': 0.40015930978245184},
 '5e39394440413c24a5becb59': {'name': '最强医圣',
  'author': '左耳思念',
  'sim': 0.4478797961933431},
 '5e39396240413c24a5becb5c': {'name': '熊孩子之穿越万界搞事情',
  'author': '抚宇',
  'sim': 0.388432640627413},
 '5e39396240413c24a5becb63'

In [28]:
coll.find_one({"name": "学神传革命"})

{'_id': ObjectId('5e3938af40413c24a5becb03'),
 'name': '学神传革命',
 'url': 'http://www.xqishuta.com/Shtml67594.html',
 'author': '老坛酸柠檬牛肉面',
 'category': '玄幻奇幻',
 'abstract': '\n    钢铁高中的学霸张百元穿越了在这个世界里，他发现自己的智商明显不够用，不过还好有一个学神培养的系统...是瞧不起我随身老爷爷吗？\n声明：本站提供《学神传革命》TXT全集下载收集整理自网络，仅作为阅读交流之用，并无任何商业目的。版权归作者或出版社所有。如作者、出版社认为本站行为侵权，请联系本站，本站会立即删除您认为侵权的作品。\n',
 'download_url': 'http://dzs.xqishuta.com/67/67594/学神传革命.txt',
 'is_downloaded': True,
 'success': True,
 'is_segment': True,
 'is_saved': True}

In [29]:
get_most_similar(ObjectId('5e3938af40413c24a5becb03')) 

{'5e39388340413c24a5becaf5': {'name': '踏破太古',
  'author': '易水川',
  'sim': 0.12196798136841969},
 '5e39388f40413c24a5becafc': {'name': '四宇沉浮',
  'author': '逍遥变',
  'sim': 0.13271231509320086},
 '5e39388f40413c24a5becafd': {'name': '我是真不想穿越',
  'author': '碧蓝的世界',
  'sim': 0.1332836793707665},
 '5e3938af40413c24a5becb02': {'name': '山海临城',
  'author': '帘里秋风',
  'sim': 0.13001000482877922},
 '5e3938c940413c24a5becb16': {'name': '天武剑界',
  'author': '武之凉夏',
  'sim': 0.12383748008224199},
 '5e3938c940413c24a5becb1c': {'name': '她有一间时空小屋',
  'author': '蜀椒',
  'sim': 0.1316201910157079},
 '5e39390d40413c24a5becb2a': {'name': '情不知所深',
  'author': '文小文呀',
  'sim': 0.12478823457285904},
 '5e39392240413c24a5becb37': {'name': '地球穿越时代',
  'author': '星殒落',
  'sim': 0.13801131815140225},
 '5e39392240413c24a5becb3c': {'name': '封灵星神',
  'author': '隔海凝望',
  'sim': 0.12437636378433749},
 '5e39393540413c24a5becb49': {'name': '异事笔录',
  'author': '溪水游',
  'sim': 0.12884894215562726},
 '5e39393540413c24a5becb4b'

### Step4: KD-Tree最近邻查询

In [30]:
import numpy as np

In [31]:
from sklearn.preprocessing import normalize

# 归一化: 归一化后相似度就是向量的点乘
X_normalized = normalize(X, norm='l2', axis=1)

In [32]:
a1 = X_normalized[1].toarray()[0]

In [33]:
a2 = X_normalized[2].toarray()[0]

In [34]:
np.dot(a1, a2)   # 结果和compute_similarity(novels[1]["_id"], novels[2]["_id"])相同

0.08939962935103601

归一化后，距离越近的向量相似度也就越高。可以想象一个球的场景

In [35]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=30, algorithm='kd_tree')

In [36]:
neigh.fit(X_normalized.toarray()) 

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=30, p=2,
                 radius=1.0)

In [37]:
neigh.kneighbors(X_normalized[1].toarray(), 30, return_distance=False)[0]

array([  1, 120, 177,  85, 160,  61, 147, 124,  33,  74,  56,  97,  89,
        39,  99,  27, 119, 122,  34, 228, 107, 109, 137, 114, 173, 139,
       105, 207,  66, 166])

In [38]:
#  测试最近邻
for i in neigh.kneighbors(X_normalized[1].toarray(), 30, return_distance=False)[0]:
    a = X_normalized[i].toarray()[0]
    print(i, np.dot(a1,a))

1 0.9999999999999999
120 0.8531018961498069
177 0.12968267376086304
85 0.1277630949123581
160 0.12707087648492058
61 0.12643771776295135
147 0.1239907810895055
124 0.12286614158923734
33 0.1219159153531192
74 0.12178731595511892
56 0.12121375393427333
97 0.1198669776436285
89 0.11942072103125408
39 0.11865805722452867
99 0.11857062688206638
27 0.11616060843702344
119 0.11579764695025005
122 0.11443666865316857
34 0.11303066289887295
228 0.11280581527571643
107 0.1124388901249547
109 0.11113704039975357
137 0.1094182653632173
114 0.10910243461641425
173 0.10839079295121082
139 0.10828408839960511
105 0.10772202882292112
207 0.10765482037448752
66 0.10711078711175559
166 0.1070823612778973


### Step5: 相似度更新到数据库

In [39]:
import operator

def get_most_similar_with_kdtree(index):
    """ 通过训练好的NearestNeighbors获取最相似的小说 """
    x = X_normalized[index].toarray()
    
    similarities = []
    for i in neigh.kneighbors(x, 30, return_distance=False)[0]:
        if i != index:
            novel = novels[i]
            a = X_normalized[i].toarray()[0]
            similarities.append({
                "id": str(novel["_id"]),
                "name": novel["name"],
                "author": novel["author"],
                "similarity":  np.dot(x[0],a)
            })
    
    return sorted(similarities, key=operator.itemgetter('similarity'), reverse=True)  

In [43]:
test_id = 0
for i, n in enumerate(novels):
    if n["name"] == "开局就是神兽":
        test_id = i
print(test_id)

219


In [45]:
get_most_similar_with_kdtree(test_id)

[{'id': '5e39396240413c24a5becb63',
  'name': '通天鸿徒',
  'author': '苏D健身',
  'similarity': 0.45367220019503995},
 {'id': '5e39394440413c24a5becb59',
  'name': '最强医圣',
  'author': '左耳思念',
  'similarity': 0.4478797961933407},
 {'id': '5e3938c940413c24a5becb16',
  'name': '天武剑界',
  'author': '武之凉夏',
  'similarity': 0.4388509209071198},
 {'id': '5e39399e40413c24a5becb87',
  'name': '无敌疯狂兑换系统',
  'author': '疯狂升级',
  'similarity': 0.43099081124232086},
 {'id': '5e3938c940413c24a5becb18',
  'name': '尘梦问逍遥',
  'author': '游梦凡人',
  'similarity': 0.4291114220849604},
 {'id': '5e39388f40413c24a5becafd',
  'name': '我是真不想穿越',
  'author': '碧蓝的世界',
  'similarity': 0.4279563904366516},
 {'id': '5e39388f40413c24a5becafe',
  'name': '辽辽天地间',
  'author': '人生大乱',
  'similarity': 0.4214504454817312},
 {'id': '5e3939e840413c24a5becbaa',
  'name': '重生之狂宠倾城魔妃',
  'author': '福来嘻嘻',
  'similarity': 0.40937902586132824},
 {'id': '5e393a0f40413c24a5becbc5',
  'name': '我的异能是雷霆',
  'author': '七茅山人',
  'similarity': 0

In [46]:
for i, n in enumerate(novels):
    print(i+1, n["name"])
    similarities = get_most_similar_with_kdtree(i)
    coll.update({'_id': n["_id"]}, {
        '$set': {
            'similarities': similarities,
            'is_compute': True
        },
    })

1 太古神狱
2 魔能星海
3 我铸造了仙界
4 当斗罗大陆回到八千年前
5 镇鼎


/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys


6 最强真言道统
7 全能召唤师系统
8 万古帝尊
9 从今天开始捡属性
10 美男天师联盟


/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys


11 不可名状的邪神
12 天苍神帝
13 武破九荒
14 混沌天帝诀
15 异世之水浒群星降临


...

/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys


227 逆世妖尊
228 中州风云记
229 武极逍遥
230 从大话西游开始打穿西游
231 我的人偶钢铁侠


/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys


232 万古最强部落
233 重生一世再续缘
234 都市之终极奶爸
235 奶爸的主神餐厅


/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
/home/gz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys
